In [1]:
# backtest_dma_2green_grid_threaded.py
"""
Threaded grid-search backtester for DMA(10,20) + strict 2-green strategy with Exit v2 + MAX HOLD 10 DAYS + optional HSL.

Features:
 - Warm cache (parquet preferred, CSV fallback) using yfinance daily data
 - Robust column normalization & duplicate handling
 - Entry: 10DMA crosses above 20DMA AND two consecutive green candles where 2nd candle's O/H/L/C > 1st
 - Exit v2:
     * Two consecutive red candles AND second closes below 20DMA, OR
     * (optional) protective exit: any close below 20DMA, OR
     * **Time exit:** if position has been held for >= 10 bars (days), OR
     * **Hard Stop-Loss (HSL):** price falls by X% from entry (e.g., 5%)
 - Optional indicator filters (RSI/MACD/ADX/SMA/Bollinger) combined via 'all' or 'any'
 - Groww-like fees + GST + STT + DP + slippage
 - Isolated capital_per_trade sizing (e.g., ₹50,000 per trade)
 - Grid search across indicator thresholds, protective_exit toggle, and optional HSL
 - Threaded over parameter combos or tickers
"""

import os
import time
import pathlib
import itertools
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import numpy as np
import yfinance as yf

# -------------------- CONFIG --------------------
TICKER_LIST = ['BSE.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CHOLAFIN.NS', 'COFORGE.NS', 'DIVISLAB.NS', 'DIXON.NS', 'NYKAA.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'ICICIBANK.NS', 'INDHOTEL.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MUTHOOTFIN.NS', 'PAYTM.NS', 'PERSISTENT.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SRF.NS', 'SHREECEM.NS', 'SOLARINDS.NS', 'TVSMOTOR.NS', 'UNITDSPR.NS']

START_DATE = "2015-01-01"
END_DATE = "2025-01-01"

DMA_FAST = 10
DMA_SLOW = 20

# trade behavior defaults
DEFAULT_TRAILING_STOP_PCT = 0.0   # not used by default here; kept for extension
DEFAULT_HARD_STOP_PCT = 5.0       # default HSL percent (e.g., 5% below entry); can be overridden per combo
DEFAULT_MAX_HOLD_DAYS = 10        # hard time-based exit at 10 days

# grid results / output
GRID_RESULTS_CSV = "dma2green_grid_results_with_fees.csv"
SAVE_DETAILED_TRADES = True
DETAILED_DIR = "dma2green_grid_runs"

# threading / parallelization
NUM_WORKERS = min(8, (os.cpu_count() or 1))
PARALLELIZE = 'combos'  # 'combos' or 'tickers'

# warm cache before threaded run? (recommended True)
WARM_CACHE = True

# isolated capital per trade (INR)
CAPITAL_PER_TRADE = 50000.0

# -------------------- FEES & SLIPPAGE (Groww-style) --------------------
SLIPPAGE_PCT = 0.05  # percent per side (adverse slippage)
# Groww brokerage: "₹20 OR 0.1% per executed order — whichever is lower, minimum ₹5"
BROKERAGE_CAP_RUPEES = 20.0
BROKERAGE_PCT_CAP = 0.1  # percent
BROKERAGE_MIN_RUPEES = 5.0

# regulatory/exchange charges (approx values; tune per your broker's live sheet)
EXCHANGE_TXN_PCT = 0.00297  # percent per side
SEBI_TURNOVER_PCT = 0.0001  # percent per side
IPFT_PCT = 0.0001          # percent per side

# STT for delivery sell
STT_SELL_PCT = 0.025  # percent on sell notional (delivery)

# stamp duty (example small state)
STAMP_DUTY_BUY_PCT = 0.003  # percent on buy notional

# DP charge
DP_CHARGE_SELL = 16.5  # rupees per sell (delivery) if notional >= 100

# GST
GST_PCT = 18.0

# Protective exit toggle will also be swept in grid
PROTECTIVE_EXIT_DEFAULT = True

# -------------------- GRID SEARCH SPACE --------------------
GRID_SEARCH_SPACE = {
    # indicator filter toggles & thresholds
    "rsi_enabled": [True, False],
    "rsi_min": [45, 50],          # require RSI >= rsi_min on entry
    "rsi_exit_enabled": [False],  # optional exit via RSI (kept off by default)

    "adx_enabled": [True, False],
    "adx_min": [18, 20, 25],

    "bb_enabled": [False, True],  # price above mid-band
    "bb_length": [20],
    "bb_stddev": [2.0],

    "sma_enabled": [False, True],
    "sma_length": [30, 50],       # require Close >= SMA_x on entry

    "macd_enabled": [False],      # optional; off by default
    "macd_fast": [12],
    "macd_slow": [26],
    "macd_signal": [9],

    "combination_mode": ['all', 'any'],

    # protective exit sweep
    "protective_exit": [True, False],

    # NEW: Hard Stop-Loss sweep
    "use_hard_stop": [True, False],
    "hard_stop_pct": [5.0]  # percent below entry (approx "5% of capital")
}
# --------------------------------------------------


# -------------------- CACHE HELPERS (parquet preferred, csv fallback) --------------------
CACHE_DIR = "cache"
pathlib.Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

_parquet_engine = None
try:
    import pyarrow  # noqa: F401
    _parquet_engine = "pyarrow"
except Exception:
    try:
        import fastparquet  # noqa: F401
        _parquet_engine = "fastparquet"
    except Exception:
        _parquet_engine = None

if _parquet_engine:
    print(f"Cache: using parquet engine '{_parquet_engine}'.")
else:
    print("Cache: parquet engine not found — falling back to CSV cache (no extra deps required).")

def ensure_cache_dir():
    pathlib.Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

def _cache_path_for_ticker(ticker, use_parquet):
    safe = ticker.replace('/', '_').replace(':', '_')
    if use_parquet:
        return os.path.join(CACHE_DIR, f"{safe}.parquet")
    else:
        return os.path.join(CACHE_DIR, f"{safe}.csv")

def download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2):
    attempt = 0
    while attempt < max_retries:
        try:
            df = yf.download(ticker, start=start, interval=interval, end=end,
                             auto_adjust=True, progress=False, threads=True, multi_level_index=False)
            return df
        except Exception as e:
            attempt += 1
            wait = backoff_sec * attempt
            print(f"  download error for {ticker} (attempt {attempt}/{max_retries}): {e}. retrying in {wait}s")
            time.sleep(wait)
    print(f"  download failed for {ticker} after {max_retries} attempts.")
    return None

def _read_cache(ticker):
    use_parquet = _parquet_engine is not None
    path = _cache_path_for_ticker(ticker, use_parquet)
    alt = _cache_path_for_ticker(ticker, not use_parquet)

    paths_to_try = []
    if os.path.exists(path):
        paths_to_try.append(path)
    if os.path.exists(alt):
        paths_to_try.append(alt)

    for p in paths_to_try:
        try:
            if os.path.getsize(p) == 0:
                print(f"  Warning: cache file {p} is zero bytes — removing.")
                try:
                    os.remove(p)
                except Exception as e:
                    print(f"    Could not remove zero-byte file {p}: {e}")
                continue
        except OSError:
            continue

        try:
            if p.endswith('.parquet'):
                return pd.read_parquet(p, engine=_parquet_engine)
            else:
                return pd.read_csv(p, index_col=0, parse_dates=True)
        except Exception as e:
            print(f"  Warning: failed to read cache for {ticker} at {p}: {e}")
            try:
                os.remove(p)
                print(f"    Removed corrupted cache file {p}.")
            except Exception as rm_e:
                print(f"    Could not remove corrupted cache file {p}: {rm_e}")
            continue
    return None

def _write_cache(ticker, df):
    use_parquet = _parquet_engine is not None
    path = _cache_path_for_ticker(ticker, use_parquet)
    tmp_path = path + ".tmp"
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        if use_parquet:
            df.to_parquet(tmp_path, engine=_parquet_engine)
        else:
            df.to_csv(tmp_path)
        os.replace(tmp_path, path)
        return True
    except Exception as e:
        print(f"  Warning: failed to write cache for {ticker} to {path}: {e}")
        try:
            if os.path.exists(tmp_path):
                os.remove(tmp_path)
        except Exception:
            pass
        if use_parquet:
            try:
                alt = _cache_path_for_ticker(ticker, use_parquet=False)
                tmp_alt = alt + ".tmp"
                df.to_csv(tmp_alt)
                os.replace(tmp_alt, alt)
                print(f"  Wrote CSV fallback cache for {ticker} to {alt}")
                return True
            except Exception as e2:
                print(f"  Warning: fallback CSV write also failed for {ticker}: {e2}")
        return False

def warm_cache(tickers, start=START_DATE, end=END_DATE):
    print(f"Warm cache: downloading {len(tickers)} tickers sequentially ...")
    ensure_cache_dir()
    for t in tickers:
        try:
            existing = _read_cache(t)
            if existing is not None:
                continue
            df = download_with_retry(t, start, end, interval="1d", max_retries=3, backoff_sec=2)
            if df is None or df.empty:
                print(f"  warm_cache: no data for {t}; skipping.")
                continue
            df = normalize_df_columns(df)
            df = collapse_duplicate_columns_take_first(df)
            ok = _write_cache(t, df)
            if not ok:
                print(f"  warm_cache: failed to write cache for {t}")
            else:
                print(f"  warm_cache: cached {t}")
            time.sleep(0.25)
        except Exception as e:
            print(f"  warm_cache: error for {t}: {e}\n{traceback.format_exc()}")
            continue
    print("Warm cache: done.")

# -------------------- NORMALIZATION & DUPLICATE HANDLING --------------------
def normalize_df_columns(df):
    if hasattr(df, "columns") and getattr(df.columns, "nlevels", 1) > 1:
        df.columns = ["_".join([str(c) for c in col if c is not None]).strip() for c in df.columns.values]

    cols = list(df.columns)
    mapping = {}
    lower_map = {c.lower(): c for c in cols}
    for name in ['close', 'high', 'low', 'open', 'volume']:
        if name in lower_map:
            mapping[lower_map[name]] = name.capitalize()
        else:
            match = next((c for c in cols if name in c.lower()), None)
            if match:
                mapping[match] = name.capitalize()
    if mapping:
        df = df.rename(columns=mapping)
    return df

def collapse_duplicate_columns_take_first(df):
    if df.columns.duplicated().any():
        dup_names = list({c for c in df.columns[df.columns.duplicated()]})
        print(f"  Warning: duplicate columns found and collapsed for: {dup_names}")
        df = df.groupby(df.columns, axis=1).first()
    return df

# -------------------- INDICATORS --------------------
def sma(series, length):
    return series.rolling(length).mean()

def rsi(df, length=14, column='Close'):
    series = df[column]
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/length, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/length, adjust=False).mean()
    rs = avg_gain / (avg_loss.replace(0, np.nan))
    df['RSI'] = (100 - (100 / (1 + rs))).fillna(50)
    return df

def macd(df, fast=12, slow=26, signal=9, column='Close'):
    ema_fast = df[column].ewm(span=fast, adjust=False).mean()
    ema_slow = df[column].ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    df['MACD'] = macd_line
    df['MACD_signal'] = signal_line
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    return df

def adx(df, n=14):
    high = df['High']; low = df['Low']; close = df['Close']
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    up_move = high.diff(); down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr_smooth = pd.Series(tr).rolling(window=n).sum()
    plus_dm_smooth = pd.Series(plus_dm).rolling(window=n).sum()
    minus_dm_smooth = pd.Series(minus_dm).rolling(window=n).sum()
    plus_di = 100 * (plus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    minus_di = 100 * (minus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)).replace([np.inf, -np.inf], 0) * 100
    df['+DI'] = plus_di; df['-DI'] = minus_di; df['ADX'] = dx.rolling(window=n).mean()
    return df

def bbands(df, length=20, stddev=2.0, column='Close'):
    mid = df[column].rolling(length).mean()
    std = df[column].rolling(length).std()
    df['BB_middle'] = mid
    df['BB_upper'] = mid + stddev*std
    df['BB_lower'] = mid - stddev*std
    return df

# -------------------- STRATEGY BUILDING BLOCKS --------------------
def two_green_strict(df):
    # True at t if t-1 and t are green, and candle t has O/H/L/C > candle t-1 O/H/L/C
    o, h, l, c = df['Open'], df['High'], df['Low'], df['Close']
    green_prev = df['Close'].shift(1) > df['Open'].shift(1)
    green_now  = df['Close'] > df['Open']
    cond_strict = (o > o.shift(1)) & (h > h.shift(1)) & (l > l.shift(1)) & (c > c.shift(1))
    return (green_prev & green_now & cond_strict)

def two_red(df):
    red_prev = df['Close'].shift(1) < df['Open'].shift(1)
    red_now  = df['Close'] < df['Open']
    return (red_prev & red_now)

def build_base_signals(df, dma_fast=DMA_FAST, dma_slow=DMA_SLOW):
    df['DMA_fast'] = sma(df['Close'], dma_fast)
    df['DMA_slow'] = sma(df['Close'], dma_slow)
    df['dma_cross_up'] = (df['DMA_fast'] > df['DMA_slow']) & (df['DMA_fast'].shift(1) <= df['DMA_slow'].shift(1))
    df['two_green_strict'] = two_green_strict(df)
    df['two_red'] = two_red(df)
    return df

# -------------------- DATA PREP (cache-aware) --------------------
def get_stock_data(ticker, start, end, indicator_params):
    ensure_cache_dir()
    raw_df = _read_cache(ticker)
    if raw_df is None:
        raw_df = download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2)
        if raw_df is None or raw_df.empty:
            return None
        raw_df = normalize_df_columns(raw_df)
        raw_df = collapse_duplicate_columns_take_first(raw_df)
        ok = _write_cache(ticker, raw_df)
        if not ok:
            print(f"  Warning: failed to cache {ticker} (continuing without cache).")

    df = raw_df.copy()
    df = normalize_df_columns(df)
    df = collapse_duplicate_columns_take_first(df)

    for c in ['Close', 'High', 'Low', 'Open', 'Volume']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    df = build_base_signals(df, dma_fast=DMA_FAST, dma_slow=DMA_SLOW)

    if indicator_params.get('rsi_enabled', False):
        df = rsi(df, length=indicator_params.get('rsi_length', 14))
    if indicator_params.get('macd_enabled', False):
        df = macd(df, fast=indicator_params.get('macd_fast', 12),
                     slow=indicator_params.get('macd_slow', 26),
                     signal=indicator_params.get('macd_signal', 9))
    if indicator_params.get('adx_enabled', False):
        df = adx(df, n=indicator_params.get('adx_length', 14))
    if indicator_params.get('sma_enabled', False):
        df[f"SMA_{indicator_params.get('sma_length', 50)}"] = df['Close'].rolling(indicator_params.get('sma_length', 50)).mean()
    if indicator_params.get('bb_enabled', False):
        df = bbands(df, length=indicator_params.get('bb_length', 20), stddev=indicator_params.get('bb_stddev', 2.0))

    df.dropna(inplace=True)
    if df.empty:
        return None
    return df

# -------------------- FEE MODEL (Groww) & slippage helpers --------------------
def groww_brokerage_for_order(order_value):
    pct_based = (BROKERAGE_PCT_CAP / 100.0) * order_value
    capped = min(BROKERAGE_CAP_RUPEES, pct_based)
    return max(capped, BROKERAGE_MIN_RUPEES)

def apply_fees_and_slippage_groww(entry_price, exit_price, position_size=1.0, is_delivery=True):
    """
    Returns:
      entry_eff, exit_eff,
      gross_return_pct, net_return_pct,
      fees_total, fees_breakdown,
      gross_profit_currency, net_profit_currency
    """
    entry_eff = entry_price * (1.0 + SLIPPAGE_PCT / 100.0)
    exit_eff = exit_price * (1.0 - SLIPPAGE_PCT / 100.0)

    entry_notional_eff = entry_eff * position_size
    exit_notional_eff = exit_eff * position_size

    gross_profit = exit_notional_eff - entry_notional_eff
    gross_return_pct = (gross_profit / entry_notional_eff) * 100.0 if entry_notional_eff != 0 else 0.0

    brokerage_entry = groww_brokerage_for_order(entry_notional_eff)
    brokerage_exit = groww_brokerage_for_order(exit_notional_eff)

    exch_entry = (EXCHANGE_TXN_PCT / 100.0) * entry_notional_eff
    exch_exit = (EXCHANGE_TXN_PCT / 100.0) * exit_notional_eff

    sebi_entry = (SEBI_TURNOVER_PCT / 100.0) * entry_notional_eff
    sebi_exit = (SEBI_TURNOVER_PCT / 100.0) * exit_notional_eff

    ipft_entry = (IPFT_PCT / 100.0) * entry_notional_eff
    ipft_exit = (IPFT_PCT / 100.0) * exit_notional_eff

    stamp_buy = (STAMP_DUTY_BUY_PCT / 100.0) * entry_notional_eff if is_delivery else 0.0
    stt_sell = (STT_SELL_PCT / 100.0) * exit_notional_eff if is_delivery else 0.0
    dp_sell = DP_CHARGE_SELL if (is_delivery and exit_notional_eff >= 100.0) else 0.0

    taxable = (brokerage_entry + brokerage_exit +
               exch_entry + exch_exit +
               ipft_entry + ipft_exit +
               sebi_entry + sebi_exit +
               dp_sell)

    gst = (GST_PCT / 100.0) * taxable

    fees_total = (brokerage_entry + brokerage_exit +
                  exch_entry + exch_exit +
                  sebi_entry + sebi_exit +
                  ipft_entry + ipft_exit +
                  stamp_buy + stt_sell + dp_sell + gst)

    net_profit = gross_profit - fees_total
    net_return_pct = (net_profit / entry_notional_eff) * 100.0 if entry_notional_eff != 0 else 0.0

    fees_breakdown = {
        "brokerage_entry": brokerage_entry,
        "brokerage_exit": brokerage_exit,
        "exchange_entry": exch_entry,
        "exchange_exit": exch_exit,
        "sebi_entry": sebi_entry,
        "sebi_exit": sebi_exit,
        "ipft_entry": ipft_entry,
        "ipft_exit": ipft_exit,
        "stamp_buy": stamp_buy,
        "stt_sell": stt_sell,
        "dp_sell": dp_sell,
        "gst": gst,
        "fees_total": fees_total
    }

    return (entry_eff, exit_eff,
            gross_return_pct, net_return_pct,
            fees_total, fees_breakdown,
            gross_profit, net_profit)

# -------------------- SIGNALS & BACKTEST (capital-per-trade sizing) --------------------
def indicator_entry_filter(row, params):
    checks = {}
    if params.get('rsi_enabled', False):
        rsi_min = params.get('rsi_min', 45)
        r = row.get('RSI', np.nan)
        checks['rsi'] = (r >= rsi_min)
    if params.get('adx_enabled', False):
        adx_min = params.get('adx_min', 20)
        adx_val = row.get('ADX', 0); plus = row.get('+DI', 0); minus = row.get('-DI', 0)
        checks['adx'] = (adx_val >= adx_min) and (plus > minus)
    if params.get('sma_enabled', False):
        sma_len = params.get('sma_length', 50)
        sma_col = f"SMA_{sma_len}"
        checks['sma'] = (row['Close'] >= row.get(sma_col, np.nan))
    if params.get('bb_enabled', False):
        # require price above mid-band
        checks['bb'] = (row['Close'] >= row.get('BB_middle', np.nan))
    if params.get('macd_enabled', False):
        checks['macd'] = (row.get('MACD', 0) > row.get('MACD_signal', 0)) and (row.get('MACD_hist', 0) > 0)
    return checks

def combine_indicator_signals(ind_results, mode='all'):
    if not ind_results:
        return True
    vals = list(ind_results.values())
    return all(vals) if mode == 'all' else any(vals)

def run_backtest_on_df(df, ticker, params, capital_per_trade=CAPITAL_PER_TRADE):
    in_position = False
    trades = []
    entry_price = 0; entry_date = None
    current_position_size = 0
    days_in_trade = 0

    protective_exit = params.get('protective_exit', PROTECTIVE_EXIT_DEFAULT)
    use_hsl = bool(params.get('use_hard_stop', False))
    hsl_pct = float(params.get('hard_stop_pct', DEFAULT_HARD_STOP_PCT))

    hard_stop_price = None

    for i in range(1, len(df)):
        prev_row = df.iloc[i-1]; current_row = df.iloc[i]
        # Base entry: DMA cross up AND strict 2-green
        base_entry = bool(current_row['dma_cross_up'] and current_row['two_green_strict'])

        # Filters
        ind_results = indicator_entry_filter(current_row, params)
        ind_combined = combine_indicator_signals(ind_results, mode=params.get('combination_mode','all'))
        enabled_any = any([params.get(k) for k in ['rsi_enabled','macd_enabled','adx_enabled','sma_enabled','bb_enabled']])
        entry_signal = base_entry and (ind_combined if enabled_any else True)

        # Enter
        if (not in_position) and entry_signal:
            entry_price = current_row['Close']
            expected_entry_eff = entry_price * (1.0 + SLIPPAGE_PCT / 100.0)
            position_size = int(capital_per_trade // expected_entry_eff)
            if position_size <= 0:
                continue
            in_position = True
            current_position_size = position_size
            entry_date = current_row.name
            days_in_trade = 0

            if use_hsl:
                hard_stop_price = entry_price * (1.0 - hsl_pct/100.0)
            else:
                hard_stop_price = None

        elif in_position:
            days_in_trade += 1

            # Exit conditions (priority: HSL -> Time -> TwoRedBelow20 -> Protective)
            exit_price = 0.0
            exit_reason = None

            # 1) Hard stop-loss
            if use_hsl and hard_stop_price is not None and (current_row['Low'] <= hard_stop_price):
                exit_price = hard_stop_price
                exit_reason = "HSL"

            # 2) Time-based exit
            elif days_in_trade >= DEFAULT_MAX_HOLD_DAYS:
                exit_price = current_row['Close']
                exit_reason = "Time"

            else:
                # 3) Exit v2 (two red and second close < 20DMA)
                exit_two_red_below_20 = bool(current_row['two_red'] and (current_row['Close'] < current_row['DMA_slow']))
                # 4) Protective exit (any close < 20DMA)
                exit_protective = bool(current_row['Close'] < current_row['DMA_slow']) if protective_exit else False

                if exit_two_red_below_20:
                    exit_price = current_row['Close']; exit_reason = "TwoRedBelow20"
                elif exit_protective:
                    exit_price = current_row['Close']; exit_reason = "Protective"

            if exit_price > 0:
                position_size = current_position_size

                (entry_eff, exit_eff,
                 gross_ret_pct_per_position, net_ret_pct_per_position,
                 fees_total, fees_breakdown,
                 gross_profit_currency, net_profit_currency) = apply_fees_and_slippage_groww(
                    entry_price, exit_price, position_size=position_size, is_delivery=True
                )

                entry_notional = entry_eff * position_size
                exit_notional = exit_eff * position_size
                roi_pct_on_capital = (net_profit_currency / capital_per_trade) * 100.0

                trades.append({
                    "ticker": ticker,
                    "entry_date": entry_date, "exit_date": current_row.name,
                    "position_size": position_size,
                    "entry_price": entry_price, "exit_price": exit_price,
                    "entry_price_effective": entry_eff, "exit_price_effective": exit_eff,
                    "entry_notional": entry_notional, "exit_notional": exit_notional,
                    "gross_profit": gross_profit_currency,
                    "net_profit": net_profit_currency,
                    "gross_return_%": gross_ret_pct_per_position,
                    "net_return_%": net_ret_pct_per_position,
                    "return_%": roi_pct_on_capital,
                    "fees": fees_total,
                    "fees_breakdown": fees_breakdown,
                    "exit_reason": exit_reason,
                    "days_held": days_in_trade
                })
                in_position = False
                entry_price = 0; entry_date = None
                current_position_size = 0
                days_in_trade = 0
                hard_stop_price = None

    return trades

# -------------------- GRID ENGINE --------------------
def generate_param_combinations(space):
    keys = list(space.keys()); vals = [space[k] for k in keys]
    combos = []
    for prod in itertools.product(*vals):
        combos.append(dict(zip(keys, prod)))
    return combos

def build_indicator_params_from_combo(combo):
    return {
        "rsi_enabled": bool(combo.get('rsi_enabled', False)),
        "rsi_min": combo.get('rsi_min', 45),
        "rsi_exit_enabled": bool(combo.get('rsi_exit_enabled', False)),

        "macd_enabled": bool(combo.get('macd_enabled', False)),
        "macd_fast": combo.get('macd_fast', 12),
        "macd_slow": combo.get('macd_slow', 26),
        "macd_signal": combo.get('macd_signal', 9),

        "adx_enabled": bool(combo.get('adx_enabled', False)),
        "adx_length": combo.get('adx_length', 14),
        "adx_min": combo.get('adx_min', 20),

        "sma_enabled": bool(combo.get('sma_enabled', False)),
        "sma_length": combo.get('sma_length', 50),

        "bb_enabled": bool(combo.get('bb_enabled', False)),
        "bb_length": combo.get('bb_length', 20),
        "bb_stddev": combo.get('bb_stddev', 2.0),

        "combination_mode": combo.get('combination_mode', 'all'),

        "protective_exit": bool(combo.get('protective_exit', PROTECTIVE_EXIT_DEFAULT)),

        # NEW: HSL
        "use_hard_stop": bool(combo.get('use_hard_stop', False)),
        "hard_stop_pct": float(combo.get('hard_stop_pct', DEFAULT_HARD_STOP_PCT))
    }

def aggregate_trades_metrics(trades, capital_per_trade=CAPITAL_PER_TRADE):
    if not trades:
        return {
            "total_trades": 0, "win_rate": 0.0, "avg_roi_pct": 0.0,
            "total_net_profit": 0.0, "avg_net_profit": 0.0, "avg_holding": 0.0,
            "overall_roi_pct": 0.0,
            "two_red_exits": 0, "protective_exits": 0, "time_exits": 0, "hsl_exits": 0
        }
    df = pd.DataFrame(trades)
    total_trades = len(df)
    wins = df[df['net_profit'] > 0]
    win_rate = len(wins) / total_trades * 100.0 if total_trades else 0.0

    avg_roi_pct = df['return_%'].mean() if total_trades else 0.0
    total_net_profit = df['net_profit'].sum() if total_trades else 0.0
    avg_net_profit = df['net_profit'].mean() if total_trades else 0.0
    avg_holding = (df['days_held'] if 'days_held' in df.columns else (pd.to_datetime(df['exit_date']) - pd.to_datetime(df['entry_date'])).dt.days).mean() if total_trades else 0.0

    two_red_exits = int((df['exit_reason'] == 'TwoRedBelow20').sum())
    protective_exits = int((df['exit_reason'] == 'Protective').sum())
    time_exits = int((df['exit_reason'] == 'Time').sum())
    hsl_exits = int((df['exit_reason'] == 'HSL').sum())

    total_capital_allocated = capital_per_trade * total_trades
    overall_roi_pct = (total_net_profit / total_capital_allocated) * 100.0 if total_capital_allocated else 0.0

    return {
        "total_trades": int(total_trades),
        "win_rate": float(win_rate),
        "avg_roi_pct": float(avg_roi_pct),
        "total_net_profit": float(total_net_profit),
        "avg_net_profit": float(avg_net_profit),
        "avg_holding": float(avg_holding),
        "overall_roi_pct": float(overall_roi_pct),
        "two_red_exits": two_red_exits,
        "protective_exits": protective_exits,
        "time_exits": time_exits,
        "hsl_exits": hsl_exits
    }

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._-" else "_" for c in s)[:180]

def process_ticker_for_combo(ticker, params):
    try:
        df = get_stock_data(ticker, START_DATE, END_DATE, params)
    except Exception as e:
        print(f"  Error fetching {ticker}: {e}")
        return []
    if df is None:
        return []
    trades = run_backtest_on_df(df, ticker, params, capital_per_trade=CAPITAL_PER_TRADE)
    return trades

def process_combo_worker(args):
    combo_idx, combo = args
    params = build_indicator_params_from_combo(combo)
    combo_name = f"{combo_idx:04d}_" + "_".join([f"{k}={v}" for k, v in combo.items()])
    overall_trades = []

    if PARALLELIZE == 'tickers':
        with ThreadPoolExecutor(max_workers=min(6, (os.cpu_count() or 1))) as tp:
            futures = {tp.submit(process_ticker_for_combo, t, params): t for t in TICKER_LIST}
            for fut in as_completed(futures):
                try:
                    trades = fut.result()
                except Exception as e:
                    print(f"  Error in ticker worker: {e}")
                    trades = []
                if trades:
                    for tr in trades:
                        tr['combo_name'] = combo_name
                    overall_trades.extend(trades)
    else:
        for ticker in TICKER_LIST:
            try:
                df = get_stock_data(ticker, START_DATE, END_DATE, params)
            except Exception as e:
                print(f"[Worker {combo_idx}] Error fetching {ticker}: {e}")
                df = None
            if df is None:
                continue
            trades = run_backtest_on_df(df, ticker, params, capital_per_trade=CAPITAL_PER_TRADE)
            for t in trades:
                t['combo_name'] = combo_name
            overall_trades.extend(trades)

    metrics = aggregate_trades_metrics(overall_trades, capital_per_trade=CAPITAL_PER_TRADE)
    return (combo_idx, combo, metrics, overall_trades)

def run_grid_search_parallel(tickers, grid_space, save_detailed=SAVE_DETAILED_TRADES):
    combos = generate_param_combinations(grid_space)
    print(f"Generated {len(combos)} parameter combinations.")
    os.makedirs(DETAILED_DIR, exist_ok=True)

    indexed_combos = list(enumerate(combos, start=1))
    num_workers = NUM_WORKERS or (os.cpu_count() or 1)
    print(f"Launching thread pool with {num_workers} worker threads. PARALLELIZE='{PARALLELIZE}'")

    results_rows = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_combo = {executor.submit(process_combo_worker, ic): ic for ic in indexed_combos}
        for fut in as_completed(future_to_combo):
            ic = future_to_combo[fut]
            try:
                combo_idx, combo, metrics, detailed_trades = fut.result()
            except Exception as e:
                print(f"[Combo {ic[0]}] Worker raised exception: {e}")
                continue

            row = {**combo, **metrics}
            results_rows.append(row)

            if save_detailed and detailed_trades:
                combo_name = f"{combo_idx:04d}_" + "_".join([f"{k}={v}" for k, v in combo.items()])
                fname = safe_filename(combo_name) + "_trades.csv"
                outpath = os.path.join(DETAILED_DIR, fname)
                try:
                    pd.DataFrame(detailed_trades).to_csv(outpath, index=False)
                except Exception as e:
                    print(f"  Warning: failed to save detailed trades for combo {combo_idx}: {e}")

            try:
                pd.DataFrame(results_rows).to_csv(GRID_RESULTS_CSV, index=False)
            except Exception as e:
                print(f"  Warning: failed to checkpoint grid results: {e}")

            print(f"[Done combo {combo_idx}] total_trades={metrics['total_trades']} win_rate={metrics['win_rate']:.2f}% overall_roi={metrics['overall_roi_pct']:.2f}% total_net_profit={metrics['total_net_profit']:.2f}")

    return pd.DataFrame(results_rows)

# -------------------- MAIN --------------------
if __name__ == "__main__":
    print("Starting threaded grid search for DMA(10,20)+2-green with Groww fees, MAX HOLD 10 days, optional HSL, and isolated capital per trade.")
    combos = generate_param_combinations(GRID_SEARCH_SPACE)
    print(f"TOTAL combos to run: {len(combos)}")

    if WARM_CACHE:
        print("WARM_CACHE is True — warming cache sequentially before threaded run (recommended).")
        warm_cache(TICKER_LIST)

    results_df = run_grid_search_parallel(TICKER_LIST, GRID_SEARCH_SPACE, save_detailed=SAVE_DETAILED_TRADES)

    if results_df is not None and not results_df.empty:
        top_by_total = results_df.sort_values(by='total_net_profit', ascending=False).head(10)
        print("\nTop 10 combos by total_net_profit:")
        print(top_by_total[['total_net_profit', 'overall_roi_pct', 'win_rate', 'total_trades']].to_string(index=False))
        top_by_win = results_df.sort_values(by='win_rate', ascending=False).head(10)
        print("\nTop 10 combos by win_rate:")
        print(top_by_win[['win_rate', 'total_net_profit', 'overall_roi_pct', 'total_trades']].to_string(index=False))
    else:
        print("No results produced.")

    print("\nGrid search finished. Results in:", GRID_RESULTS_CSV)


Cache: using parquet engine 'pyarrow'.
Starting threaded grid search for DMA(10,20)+2-green with Groww fees, MAX HOLD 10 days, optional HSL, and isolated capital per trade.
TOTAL combos to run: 1536
WARM_CACHE is True — warming cache sequentially before threaded run (recommended).
Warm cache: downloading 30 tickers sequentially ...
  warm_cache: cached BSE.NS
  warm_cache: cached BAJFINANCE.NS
  warm_cache: cached BAJAJFINSV.NS
  warm_cache: cached BDL.NS
  warm_cache: cached BEL.NS
  warm_cache: cached BHARTIARTL.NS
  warm_cache: cached CHOLAFIN.NS
  warm_cache: cached COFORGE.NS
  warm_cache: cached DIVISLAB.NS
  warm_cache: cached DIXON.NS
  warm_cache: cached NYKAA.NS
  warm_cache: cached HDFCBANK.NS
  warm_cache: cached HDFCLIFE.NS
  warm_cache: cached ICICIBANK.NS
  warm_cache: cached INDHOTEL.NS
  warm_cache: cached INDIGO.NS
  warm_cache: cached KOTAKBANK.NS
  warm_cache: cached MFSL.NS
  warm_cache: cached MAXHEALTH.NS
  warm_cache: cached MAZDOCK.NS
  warm_cache: cached MUTHO

# ATR addition

In [ ]:
# backtest_dma_2green_grid_threaded.py
"""
Threaded grid-search backtester for DMA(10,20) + strict 2-green strategy
with Exit v2 + MAX HOLD 10 DAYS + optional HSL + optional TSL.

Features:
 - Warm cache (parquet preferred, CSV fallback) using yfinance daily data
 - Robust column normalization & duplicate handling
 - Entry: 10DMA crosses above 20DMA AND two consecutive green candles where 2nd candle's O/H/L/C > 1st
 - Exit v2:
     * Two consecutive red candles AND second closes below 20DMA, OR
     * (optional) protective exit: any close below 20DMA, OR
     * **Time exit:** if position has been held for >= 10 bars (days), OR
     * **Hard Stop-Loss (HSL):** price falls by X% from entry (e.g., 5%), OR
     * **Trailing Stop-Loss (TSL):** X% below highest high since entry (e.g., 10%)
 - Optional indicator filters (RSI/MACD/ADX/SMA/Bollinger) combined via 'all' or 'any'
 - Groww-like fees + GST + STT + DP + slippage
 - Isolated capital_per_trade sizing (e.g., ₹50,000 per trade)
 - Grid search across indicator thresholds, protective_exit toggle, and HSL/TSL toggles
 - Threaded over parameter combos or tickers
"""

import os
import time
import pathlib
import itertools
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import numpy as np
import yfinance as yf

# -------------------- CONFIG --------------------
TICKER_LIST = ['BSE.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CHOLAFIN.NS', 'COFORGE.NS', 'DIVISLAB.NS', 'DIXON.NS', 'NYKAA.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'ICICIBANK.NS', 'INDHOTEL.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MUTHOOTFIN.NS', 'PAYTM.NS', 'PERSISTENT.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SRF.NS', 'SHREECEM.NS', 'SOLARINDS.NS', 'TVSMOTOR.NS', 'UNITDSPR.NS']

START_DATE = "2015-01-01"
END_DATE = "2025-01-01"

DMA_FAST = 10
DMA_SLOW = 20

# trade behavior defaults
DEFAULT_TRAILING_STOP_PCT = 10.0  # new default TSL
DEFAULT_HARD_STOP_PCT = 5.0       # default HSL percent (e.g., 5% below entry)
DEFAULT_MAX_HOLD_DAYS = 10        # hard time-based exit at 10 days

# grid results / output
GRID_RESULTS_CSV = "dma2green_grid_results_with_fees.csv"
SAVE_DETAILED_TRADES = True
DETAILED_DIR = "dma2green_grid_runs"

# threading / parallelization
NUM_WORKERS = min(8, (os.cpu_count() or 1))
PARALLELIZE = 'combos'  # 'combos' or 'tickers'

# warm cache before threaded run? (recommended True)
WARM_CACHE = True

# isolated capital per trade (INR)
CAPITAL_PER_TRADE = 50000.0

# -------------------- FEES & SLIPPAGE (Groww-style) --------------------
SLIPPAGE_PCT = 0.05  # percent per side (adverse slippage)
# Groww brokerage: "₹20 OR 0.1% per executed order — whichever is lower, minimum ₹5"
BROKERAGE_CAP_RUPEES = 20.0
BROKERAGE_PCT_CAP = 0.1  # percent
BROKERAGE_MIN_RUPEES = 5.0

# regulatory/exchange charges (approx values; tune per your broker's live sheet)
EXCHANGE_TXN_PCT = 0.00297  # percent per side
SEBI_TURNOVER_PCT = 0.0001  # percent per side
IPFT_PCT = 0.0001          # percent per side

# STT for delivery sell
STT_SELL_PCT = 0.025  # percent on sell notional (delivery)

# stamp duty (example small state)
STAMP_DUTY_BUY_PCT = 0.003  # percent on buy notional

# DP charge
DP_CHARGE_SELL = 16.5  # rupees per sell (delivery) if notional >= 100

# GST
GST_PCT = 18.0

# Protective exit toggle will also be swept in grid
PROTECTIVE_EXIT_DEFAULT = True

# -------------------- GRID SEARCH SPACE --------------------
GRID_SEARCH_SPACE = {
    # indicator filter toggles & thresholds
    "rsi_enabled": [True, False],
    "rsi_min": [45, 50],          # require RSI >= rsi_min on entry
    "rsi_exit_enabled": [False],  # optional exit via RSI (kept off by default)

    "adx_enabled": [True, False],
    "adx_min": [18, 20, 25],

    "bb_enabled": [False, True],  # price above mid-band
    "bb_length": [20],
    "bb_stddev": [2.0],

    "sma_enabled": [False, True],
    "sma_length": [30, 50],       # require Close >= SMA_x on entry

    "macd_enabled": [False],      # optional; off by default
    "macd_fast": [12],
    "macd_slow": [26],
    "macd_signal": [9],

    "combination_mode": ['all', 'any'],

    # protective exit sweep
    "protective_exit": [True, False],

    # Hard Stop-Loss (HSL) sweep
    "use_hard_stop": [True, False],
    "hard_stop_pct": [5.0],

    # NEW: Trailing Stop-Loss (TSL) sweep
    "use_trailing_stop": [True, False],
    "trailing_stop_pct": [8.0, 10.0]  # try a couple of trailing distances
}
# --------------------------------------------------


# -------------------- CACHE HELPERS (parquet preferred, csv fallback) --------------------
CACHE_DIR = "cache"
pathlib.Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

_parquet_engine = None
try:
    import pyarrow  # noqa: F401
    _parquet_engine = "pyarrow"
except Exception:
    try:
        import fastparquet  # noqa: F401
        _parquet_engine = "fastparquet"
    except Exception:
        _parquet_engine = None

if _parquet_engine:
    print(f"Cache: using parquet engine '{_parquet_engine}'.")
else:
    print("Cache: parquet engine not found — falling back to CSV cache (no extra deps required).")

def ensure_cache_dir():
    pathlib.Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

def _cache_path_for_ticker(ticker, use_parquet):
    safe = ticker.replace('/', '_').replace(':', '_')
    if use_parquet:
        return os.path.join(CACHE_DIR, f"{safe}.parquet")
    else:
        return os.path.join(CACHE_DIR, f"{safe}.csv")

def download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2):
    attempt = 0
    while attempt < max_retries:
        try:
            df = yf.download(ticker, start=start, interval=interval, end=end,
                             auto_adjust=True, progress=False, threads=True, multi_level_index=False)
            return df
        except Exception as e:
            attempt += 1
            wait = backoff_sec * attempt
            print(f"  download error for {ticker} (attempt {attempt}/{max_retries}): {e}. retrying in {wait}s")
            time.sleep(wait)
    print(f"  download failed for {ticker} after {max_retries} attempts.")
    return None

def _read_cache(ticker):
    use_parquet = _parquet_engine is not None
    path = _cache_path_for_ticker(ticker, use_parquet)
    alt = _cache_path_for_ticker(ticker, not use_parquet)

    paths_to_try = []
    if os.path.exists(path):
        paths_to_try.append(path)
    if os.path.exists(alt):
        paths_to_try.append(alt)

    for p in paths_to_try:
        try:
            if os.path.getsize(p) == 0:
                print(f"  Warning: cache file {p} is zero bytes — removing.")
                try:
                    os.remove(p)
                except Exception as e:
                    print(f"    Could not remove zero-byte file {p}: {e}")
                continue
        except OSError:
            continue

        try:
            if p.endswith('.parquet'):
                return pd.read_parquet(p, engine=_parquet_engine)
            else:
                return pd.read_csv(p, index_col=0, parse_dates=True)
        except Exception as e:
            print(f"  Warning: failed to read cache for {ticker} at {p}: {e}")
            try:
                os.remove(p)
                print(f"    Removed corrupted cache file {p}.")
            except Exception as rm_e:
                print(f"    Could not remove corrupted cache file {p}: {rm_e}")
            continue
    return None

def _write_cache(ticker, df):
    use_parquet = _parquet_engine is not None
    path = _cache_path_for_ticker(ticker, use_parquet)
    tmp_path = path + ".tmp"
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        if use_parquet:
            df.to_parquet(tmp_path, engine=_parquet_engine)
        else:
            df.to_csv(tmp_path)
        os.replace(tmp_path, path)
        return True
    except Exception as e:
        print(f"  Warning: failed to write cache for {ticker} to {path}: {e}")
        try:
            if os.path.exists(tmp_path):
                os.remove(tmp_path)
        except Exception:
            pass
        if use_parquet:
            try:
                alt = _cache_path_for_ticker(ticker, use_parquet=False)
                tmp_alt = alt + ".tmp"
                df.to_csv(tmp_alt)
                os.replace(tmp_alt, alt)
                print(f"  Wrote CSV fallback cache for {ticker} to {alt}")
                return True
            except Exception as e2:
                print(f"  Warning: fallback CSV write also failed for {ticker}: {e2}")
        return False

def warm_cache(tickers, start=START_DATE, end=END_DATE):
    print(f"Warm cache: downloading {len(tickers)} tickers sequentially ...")
    ensure_cache_dir()
    for t in tickers:
        try:
            existing = _read_cache(t)
            if existing is not None:
                continue
            df = download_with_retry(t, start, end, interval="1d", max_retries=3, backoff_sec=2)
            if df is None or df.empty:
                print(f"  warm_cache: no data for {t}; skipping.")
                continue
            df = normalize_df_columns(df)
            df = collapse_duplicate_columns_take_first(df)
            ok = _write_cache(t, df)
            if not ok:
                print(f"  warm_cache: failed to write cache for {t}")
            else:
                print(f"  warm_cache: cached {t}")
            time.sleep(0.25)
        except Exception as e:
            print(f"  warm_cache: error for {t}: {e}\n{traceback.format_exc()}")
            continue
    print("Warm cache: done.")

# -------------------- NORMALIZATION & DUPLICATE HANDLING --------------------
def normalize_df_columns(df):
    if hasattr(df, "columns") and getattr(df.columns, "nlevels", 1) > 1:
        df.columns = ["_".join([str(c) for c in col if c is not None]).strip() for c in df.columns.values]

    cols = list(df.columns)
    mapping = {}
    lower_map = {c.lower(): c for c in cols}
    for name in ['close', 'high', 'low', 'open', 'volume']:
        if name in lower_map:
            mapping[lower_map[name]] = name.capitalize()
        else:
            match = next((c for c in cols if name in c.lower()), None)
            if match:
                mapping[match] = name.capitalize()
    if mapping:
        df = df.rename(columns=mapping)
    return df

def collapse_duplicate_columns_take_first(df):
    if df.columns.duplicated().any():
        dup_names = list({c for c in df.columns[df.columns.duplicated()]})
        print(f"  Warning: duplicate columns found and collapsed for: {dup_names}")
        df = df.groupby(df.columns, axis=1).first()
    return df

# -------------------- INDICATORS --------------------
def sma(series, length):
    return series.rolling(length).mean()

def rsi(df, length=14, column='Close'):
    series = df[column]
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/length, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/length, adjust=False).mean()
    rs = avg_gain / (avg_loss.replace(0, np.nan))
    df['RSI'] = (100 - (100 / (1 + rs))).fillna(50)
    return df

def macd(df, fast=12, slow=26, signal=9, column='Close'):
    ema_fast = df[column].ewm(span=fast, adjust=False).mean()
    ema_slow = df[column].ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    df['MACD'] = macd_line
    df['MACD_signal'] = signal_line
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    return df

def adx(df, n=14):
    high = df['High']; low = df['Low']; close = df['Close']
    tr1 = high - low
    tr2 = (high - close.shift(1)).abs()
    tr3 = (low - close.shift(1)).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    up_move = high.diff(); down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr_smooth = pd.Series(tr).rolling(window=n).sum()
    plus_dm_smooth = pd.Series(plus_dm).rolling(window=n).sum()
    minus_dm_smooth = pd.Series(minus_dm).rolling(window=n).sum()
    plus_di = 100 * (plus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    minus_di = 100 * (minus_dm_smooth / tr_smooth).replace([np.inf, -np.inf], 0).fillna(0)
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)).replace([np.inf, -np.inf], 0) * 100
    df['+DI'] = plus_di; df['-DI'] = minus_di; df['ADX'] = dx.rolling(window=n).mean()
    return df

def bbands(df, length=20, stddev=2.0, column='Close'):
    mid = df[column].rolling(length).mean()
    std = df[column].rolling(length).std()
    df['BB_middle'] = mid
    df['BB_upper'] = mid + stddev*std
    df['BB_lower'] = mid - stddev*std
    return df

# -------------------- STRATEGY BUILDING BLOCKS --------------------
def two_green_strict(df):
    # True at t if t-1 and t are green, and candle t has O/H/L/C > candle t-1 O/H/L/C
    o, h, l, c = df['Open'], df['High'], df['Low'], df['Close']
    green_prev = df['Close'].shift(1) > df['Open'].shift(1)
    green_now  = df['Close'] > df['Open']
    cond_strict = (o > o.shift(1)) & (h > h.shift(1)) & (l > l.shift(1)) & (c > c.shift(1))
    return (green_prev & green_now & cond_strict)

def two_red(df):
    red_prev = df['Close'].shift(1) < df['Open'].shift(1)
    red_now  = df['Close'] < df['Open']
    return (red_prev & red_now)

def build_base_signals(df, dma_fast=DMA_FAST, dma_slow=DMA_SLOW):
    df['DMA_fast'] = sma(df['Close'], dma_fast)
    df['DMA_slow'] = sma(df['Close'], dma_slow)
    df['dma_cross_up'] = (df['DMA_fast'] > df['DMA_slow']) & (df['DMA_fast'].shift(1) <= df['DMA_slow'].shift(1))
    df['two_green_strict'] = two_green_strict(df)
    df['two_red'] = two_red(df)
    return df

# -------------------- DATA PREP (cache-aware) --------------------
def get_stock_data(ticker, start, end, indicator_params):
    ensure_cache_dir()
    raw_df = _read_cache(ticker)
    if raw_df is None:
        raw_df = download_with_retry(ticker, start, end, interval="1d", max_retries=3, backoff_sec=2)
        if raw_df is None or raw_df.empty:
            return None
        raw_df = normalize_df_columns(raw_df)
        raw_df = collapse_duplicate_columns_take_first(raw_df)
        ok = _write_cache(ticker, raw_df)
        if not ok:
            print(f"  Warning: failed to cache {ticker} (continuing without cache).")

    df = raw_df.copy()
    df = normalize_df_columns(df)
    df = collapse_duplicate_columns_take_first(df)

    for c in ['Close', 'High', 'Low', 'Open', 'Volume']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    df = build_base_signals(df, dma_fast=DMA_FAST, dma_slow=DMA_SLOW)

    if indicator_params.get('rsi_enabled', False):
        df = rsi(df, length=indicator_params.get('rsi_length', 14))
    if indicator_params.get('macd_enabled', False):
        df = macd(df, fast=indicator_params.get('macd_fast', 12),
                     slow=indicator_params.get('macd_slow', 26),
                     signal=indicator_params.get('macd_signal', 9))
    if indicator_params.get('adx_enabled', False):
        df = adx(df, n=indicator_params.get('adx_length', 14))
    if indicator_params.get('sma_enabled', False):
        df[f"SMA_{indicator_params.get('sma_length', 50)}"] = df['Close'].rolling(indicator_params.get('sma_length', 50)).mean()
    if indicator_params.get('bb_enabled', False):
        df = bbands(df, length=indicator_params.get('bb_length', 20), stddev=indicator_params.get('bb_stddev', 2.0))

    df.dropna(inplace=True)
    if df.empty:
        return None
    return df

# -------------------- FEE MODEL (Groww) & slippage helpers --------------------
def groww_brokerage_for_order(order_value):
    pct_based = (BROKERAGE_PCT_CAP / 100.0) * order_value
    capped = min(BROKERAGE_CAP_RUPEES, pct_based)
    return max(capped, BROKERAGE_MIN_RUPEES)

def apply_fees_and_slippage_groww(entry_price, exit_price, position_size=1.0, is_delivery=True):
    """
    Returns:
      entry_eff, exit_eff,
      gross_return_pct, net_return_pct,
      fees_total, fees_breakdown,
      gross_profit_currency, net_profit_currency
    """
    entry_eff = entry_price * (1.0 + SLIPPAGE_PCT / 100.0)
    exit_eff = exit_price * (1.0 - SLIPPAGE_PCT / 100.0)

    entry_notional_eff = entry_eff * position_size
    exit_notional_eff = exit_eff * position_size

    gross_profit = exit_notional_eff - entry_notional_eff
    gross_return_pct = (gross_profit / entry_notional_eff) * 100.0 if entry_notional_eff != 0 else 0.0

    brokerage_entry = groww_brokerage_for_order(entry_notional_eff)
    brokerage_exit = groww_brokerage_for_order(exit_notional_eff)

    exch_entry = (EXCHANGE_TXN_PCT / 100.0) * entry_notional_eff
    exch_exit = (EXCHANGE_TXN_PCT / 100.0) * exit_notional_eff

    sebi_entry = (SEBI_TURNOVER_PCT / 100.0) * entry_notional_eff
    sebi_exit = (SEBI_TURNOVER_PCT / 100.0) * exit_notional_eff

    ipft_entry = (IPFT_PCT / 100.0) * entry_notional_eff
    ipft_exit = (IPFT_PCT / 100.0) * exit_notional_eff

    stamp_buy = (STAMP_DUTY_BUY_PCT / 100.0) * entry_notional_eff if is_delivery else 0.0
    stt_sell = (STT_SELL_PCT / 100.0) * exit_notional_eff if is_delivery else 0.0
    dp_sell = DP_CHARGE_SELL if (is_delivery and exit_notional_eff >= 100.0) else 0.0

    taxable = (brokerage_entry + brokerage_exit +
               exch_entry + exch_exit +
               ipft_entry + ipft_exit +
               sebi_entry + sebi_exit +
               dp_sell)

    gst = (GST_PCT / 100.0) * taxable

    fees_total = (brokerage_entry + brokerage_exit +
                  exch_entry + exch_exit +
                  sebi_entry + sebi_exit +
                  ipft_entry + ipft_exit +
                  stamp_buy + stt_sell + dp_sell + gst)

    net_profit = gross_profit - fees_total
    net_return_pct = (net_profit / entry_notional_eff) * 100.0 if entry_notional_eff != 0 else 0.0

    fees_breakdown = {
        "brokerage_entry": brokerage_entry,
        "brokerage_exit": brokerage_exit,
        "exchange_entry": exch_entry,
        "exchange_exit": exch_exit,
        "sebi_entry": sebi_entry,
        "sebi_exit": sebi_exit,
        "ipft_entry": ipft_entry,
        "ipft_exit": ipft_exit,
        "stamp_buy": stamp_buy,
        "stt_sell": stt_sell,
        "dp_sell": dp_sell,
        "gst": gst,
        "fees_total": fees_total
    }

    return (entry_eff, exit_eff,
            gross_return_pct, net_return_pct,
            fees_total, fees_breakdown,
            gross_profit, net_profit)

# -------------------- SIGNALS & BACKTEST (capital-per-trade sizing) --------------------
def indicator_entry_filter(row, params):
    checks = {}
    if params.get('rsi_enabled', False):
        rsi_min = params.get('rsi_min', 45)
        r = row.get('RSI', np.nan)
        checks['rsi'] = (r >= rsi_min)
    if params.get('adx_enabled', False):
        adx_min = params.get('adx_min', 20)
        adx_val = row.get('ADX', 0); plus = row.get('+DI', 0); minus = row.get('-DI', 0)
        checks['adx'] = (adx_val >= adx_min) and (plus > minus)
    if params.get('sma_enabled', False):
        sma_len = params.get('sma_length', 50)
        sma_col = f"SMA_{sma_len}"
        checks['sma'] = (row['Close'] >= row.get(sma_col, np.nan))
    if params.get('bb_enabled', False):
        # require price above mid-band
        checks['bb'] = (row['Close'] >= row.get('BB_middle', np.nan))
    if params.get('macd_enabled', False):
        checks['macd'] = (row.get('MACD', 0) > row.get('MACD_signal', 0)) and (row.get('MACD_hist', 0) > 0)
    return checks

def combine_indicator_signals(ind_results, mode='all'):
    if not ind_results:
        return True
    vals = list(ind_results.values())
    return all(vals) if mode == 'all' else any(vals)

def run_backtest_on_df(df, ticker, params, capital_per_trade=CAPITAL_PER_TRADE):
    in_position = False
    trades = []
    entry_price = 0; entry_date = None
    current_position_size = 0
    days_in_trade = 0

    protective_exit = params.get('protective_exit', PROTECTIVE_EXIT_DEFAULT)

    # HSL
    use_hsl = bool(params.get('use_hard_stop', False))
    hsl_pct = float(params.get('hard_stop_pct', DEFAULT_HARD_STOP_PCT))
    hard_stop_price = None

    # TSL
    use_tsl = bool(params.get('use_trailing_stop', False))
    tsl_pct = float(params.get('trailing_stop_pct', DEFAULT_TRAILING_STOP_PCT))
    peak_price = None
    trailing_stop_price = None

    for i in range(1, len(df)):
        prev_row = df.iloc[i-1]; current_row = df.iloc[i]
        # Base entry: DMA cross up AND strict 2-green
        base_entry = bool(current_row['dma_cross_up'] and current_row['two_green_strict'])

        # Filters
        ind_results = indicator_entry_filter(current_row, params)
        ind_combined = combine_indicator_signals(ind_results, mode=params.get('combination_mode','all'))
        enabled_any = any([params.get(k) for k in ['rsi_enabled','macd_enabled','adx_enabled','sma_enabled','bb_enabled']])
        entry_signal = base_entry and (ind_combined if enabled_any else True)

        # Enter
        if (not in_position) and entry_signal:
            entry_price = current_row['Close']
            expected_entry_eff = entry_price * (1.0 + SLIPPAGE_PCT / 100.0)
            position_size = int(capital_per_trade // expected_entry_eff)
            if position_size <= 0:
                continue
            in_position = True
            current_position_size = position_size
            entry_date = current_row.name
            days_in_trade = 0

            if use_hsl:
                hard_stop_price = entry_price * (1.0 - hsl_pct/100.0)
            else:
                hard_stop_price = None

            if use_tsl:
                peak_price = entry_price
                trailing_stop_price = peak_price * (1.0 - tsl_pct/100.0)
            else:
                peak_price = None
                trailing_stop_price = None

        elif in_position:
            days_in_trade += 1

            # Update TSL state first using this bar's High
            if use_tsl:
                if peak_price is None:
                    peak_price = entry_price
                if current_row['High'] > peak_price:
                    peak_price = current_row['High']
                trailing_stop_price = peak_price * (1.0 - tsl_pct/100.0)

            # Exit conditions (priority: HSL -> TSL -> Time -> TwoRedBelow20 -> Protective)
            exit_price = 0.0
            exit_reason = None

            # 1) Hard stop-loss (simulate intraday fill at stop level)
            if use_hsl and hard_stop_price is not None and (current_row['Low'] <= hard_stop_price):
                exit_price = hard_stop_price
                exit_reason = "HSL"

            # 2) Trailing stop-loss
            elif use_tsl and trailing_stop_price is not None and (current_row['Low'] <= trailing_stop_price):
                exit_price = trailing_stop_price
                exit_reason = "TSL"

            # 3) Time-based exit
            elif days_in_trade >= DEFAULT_MAX_HOLD_DAYS:
                exit_price = current_row['Close']
                exit_reason = "Time"

            else:
                # 4) Exit v2 (two red and second close < 20DMA)
                exit_two_red_below_20 = bool(current_row['two_red'] and (current_row['Close'] < current_row['DMA_slow']))
                # 5) Protective exit (any close < 20DMA)
                exit_protective = bool(current_row['Close'] < current_row['DMA_slow']) if protective_exit else False

                if exit_two_red_below_20:
                    exit_price = current_row['Close']; exit_reason = "TwoRedBelow20"
                elif exit_protective:
                    exit_price = current_row['Close']; exit_reason = "Protective"

            if exit_price > 0:
                position_size = current_position_size

                (entry_eff, exit_eff,
                 gross_ret_pct_per_position, net_ret_pct_per_position,
                 fees_total, fees_breakdown,
                 gross_profit_currency, net_profit_currency) = apply_fees_and_slippage_groww(
                    entry_price, exit_price, position_size=position_size, is_delivery=True
                )

                entry_notional = entry_eff * position_size
                exit_notional = exit_eff * position_size
                roi_pct_on_capital = (net_profit_currency / capital_per_trade) * 100.0

                trades.append({
                    "ticker": ticker,
                    "entry_date": entry_date, "exit_date": current_row.name,
                    "position_size": position_size,
                    "entry_price": entry_price, "exit_price": exit_price,
                    "entry_price_effective": entry_eff, "exit_price_effective": exit_eff,
                    "entry_notional": entry_notional, "exit_notional": exit_notional,
                    "gross_profit": gross_profit_currency,
                    "net_profit": net_profit_currency,
                    "gross_return_%": gross_ret_pct_per_position,
                    "net_return_%": net_ret_pct_per_position,
                    "return_%": roi_pct_on_capital,
                    "fees": fees_total,
                    "fees_breakdown": fees_breakdown,
                    "exit_reason": exit_reason,
                    "days_held": days_in_trade
                })
                in_position = False
                entry_price = 0; entry_date = None
                current_position_size = 0
                days_in_trade = 0
                hard_stop_price = None
                peak_price = None
                trailing_stop_price = None

    return trades

# -------------------- GRID ENGINE --------------------
def generate_param_combinations(space):
    keys = list(space.keys()); vals = [space[k] for k in keys]
    combos = []
    for prod in itertools.product(*vals):
        combos.append(dict(zip(keys, prod)))
    return combos

def build_indicator_params_from_combo(combo):
    return {
        "rsi_enabled": bool(combo.get('rsi_enabled', False)),
        "rsi_min": combo.get('rsi_min', 45),
        "rsi_exit_enabled": bool(combo.get('rsi_exit_enabled', False)),

        "macd_enabled": bool(combo.get('macd_enabled', False)),
        "macd_fast": combo.get('macd_fast', 12),
        "macd_slow": combo.get('macd_slow', 26),
        "macd_signal": combo.get('macd_signal', 9),

        "adx_enabled": bool(combo.get('adx_enabled', False)),
        "adx_length": combo.get('adx_length', 14),
        "adx_min": combo.get('adx_min', 20),

        "sma_enabled": bool(combo.get('sma_enabled', False)),
        "sma_length": combo.get('sma_length', 50),

        "bb_enabled": bool(combo.get('bb_enabled', False)),
        "bb_length": combo.get('bb_length', 20),
        "bb_stddev": combo.get('bb_stddev', 2.0),

        "combination_mode": combo.get('combination_mode', 'all'),

        "protective_exit": bool(combo.get('protective_exit', PROTECTIVE_EXIT_DEFAULT)),

        # HSL
        "use_hard_stop": bool(combo.get('use_hard_stop', False)),
        "hard_stop_pct": float(combo.get('hard_stop_pct', DEFAULT_HARD_STOP_PCT)),

        # TSL
        "use_trailing_stop": bool(combo.get('use_trailing_stop', False)),
        "trailing_stop_pct": float(combo.get('trailing_stop_pct', DEFAULT_TRAILING_STOP_PCT))
    }

def aggregate_trades_metrics(trades, capital_per_trade=CAPITAL_PER_TRADE):
    if not trades:
        return {
            "total_trades": 0, "win_rate": 0.0, "avg_roi_pct": 0.0,
            "total_net_profit": 0.0, "avg_net_profit": 0.0, "avg_holding": 0.0,
            "overall_roi_pct": 0.0,
            "two_red_exits": 0, "protective_exits": 0, "time_exits": 0, "hsl_exits": 0, "tsl_exits": 0
        }
    df = pd.DataFrame(trades)
    total_trades = len(df)
    wins = df[df['net_profit'] > 0]
    win_rate = len(wins) / total_trades * 100.0 if total_trades else 0.0

    avg_roi_pct = df['return_%'].mean() if total_trades else 0.0
    total_net_profit = df['net_profit'].sum() if total_trades else 0.0
    avg_net_profit = df['net_profit'].mean() if total_trades else 0.0
    avg_holding = (df['days_held'] if 'days_held' in df.columns else (pd.to_datetime(df['exit_date']) - pd.to_datetime(df['entry_date'])).dt.days).mean() if total_trades else 0.0

    two_red_exits = int((df['exit_reason'] == 'TwoRedBelow20').sum())
    protective_exits = int((df['exit_reason'] == 'Protective').sum())
    time_exits = int((df['exit_reason'] == 'Time').sum())
    hsl_exits = int((df['exit_reason'] == 'HSL').sum())
    tsl_exits = int((df['exit_reason'] == 'TSL').sum())

    total_capital_allocated = capital_per_trade * total_trades
    overall_roi_pct = (total_net_profit / total_capital_allocated) * 100.0 if total_capital_allocated else 0.0

    return {
        "total_trades": int(total_trades),
        "win_rate": float(win_rate),
        "avg_roi_pct": float(avg_roi_pct),
        "total_net_profit": float(total_net_profit),
        "avg_net_profit": float(avg_net_profit),
        "avg_holding": float(avg_holding),
        "overall_roi_pct": float(overall_roi_pct),
        "two_red_exits": two_red_exits,
        "protective_exits": protective_exits,
        "time_exits": time_exits,
        "hsl_exits": hsl_exits,
        "tsl_exits": tsl_exits
    }

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._-" else "_" for c in s)[:180]

def process_ticker_for_combo(ticker, params):
    try:
        df = get_stock_data(ticker, START_DATE, END_DATE, params)
    except Exception as e:
        print(f"  Error fetching {ticker}: {e}")
        return []
    if df is None:
        return []
    trades = run_backtest_on_df(df, ticker, params, capital_per_trade=CAPITAL_PER_TRADE)
    return trades

def process_combo_worker(args):
    combo_idx, combo = args
    params = build_indicator_params_from_combo(combo)
    combo_name = f"{combo_idx:04d}_" + "_".join([f"{k}={v}" for k, v in combo.items()])
    overall_trades = []

    if PARALLELIZE == 'tickers':
        with ThreadPoolExecutor(max_workers=min(6, (os.cpu_count() or 1))) as tp:
            futures = {tp.submit(process_ticker_for_combo, t, params): t for t in TICKER_LIST}
            for fut in as_completed(futures):
                try:
                    trades = fut.result()
                except Exception as e:
                    print(f"  Error in ticker worker: {e}")
                    trades = []
                if trades:
                    for tr in trades:
                        tr['combo_name'] = combo_name
                    overall_trades.extend(trades)
    else:
        for ticker in TICKER_LIST:
            try:
                df = get_stock_data(ticker, START_DATE, END_DATE, params)
            except Exception as e:
                print(f"[Worker {combo_idx}] Error fetching {ticker}: {e}")
                df = None
            if df is None:
                continue
            trades = run_backtest_on_df(df, ticker, params, capital_per_trade=CAPITAL_PER_TRADE)
            for t in trades:
                t['combo_name'] = combo_name
            overall_trades.extend(trades)

    metrics = aggregate_trades_metrics(overall_trades, capital_per_trade=CAPITAL_PER_TRADE)
    return (combo_idx, combo, metrics, overall_trades)

def run_grid_search_parallel(tickers, grid_space, save_detailed=SAVE_DETAILED_TRADES):
    combos = generate_param_combinations(grid_space)
    print(f"Generated {len(combos)} parameter combinations.")
    os.makedirs(DETAILED_DIR, exist_ok=True)

    indexed_combos = list(enumerate(combos, start=1))
    num_workers = NUM_WORKERS or (os.cpu_count() or 1)
    print(f"Launching thread pool with {num_workers} worker threads. PARALLELIZE='{PARALLELIZE}'")

    results_rows = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_combo = {executor.submit(process_combo_worker, ic): ic for ic in indexed_combos}
        for fut in as_completed(future_to_combo):
            ic = future_to_combo[fut]
            try:
                combo_idx, combo, metrics, detailed_trades = fut.result()
            except Exception as e:
                print(f"[Combo {ic[0]}] Worker raised exception: {e}")
                continue

            row = {**combo, **metrics}
            results_rows.append(row)

            if save_detailed and detailed_trades:
                combo_name = f"{combo_idx:04d}_" + "_".join([f"{k}={v}" for k, v in combo.items()])
                fname = safe_filename(combo_name) + "_trades.csv"
                outpath = os.path.join(DETAILED_DIR, fname)
                try:
                    pd.DataFrame(detailed_trades).to_csv(outpath, index=False)
                except Exception as e:
                    print(f"  Warning: failed to save detailed trades for combo {combo_idx}: {e}")

            try:
                pd.DataFrame(results_rows).to_csv(GRID_RESULTS_CSV, index=False)
            except Exception as e:
                print(f"  Warning: failed to checkpoint grid results: {e}")

            print(f"[Done combo {combo_idx}] total_trades={metrics['total_trades']} win_rate={metrics['win_rate']:.2f}% overall_roi={metrics['overall_roi_pct']:.2f}% total_net_profit={metrics['total_net_profit']:.2f}")

    return pd.DataFrame(results_rows)

# -------------------- MAIN --------------------
if __name__ == "__main__":
    print("Starting threaded grid search for DMA(10,20)+2-green with Groww fees, MAX HOLD 10 days, optional HSL & TSL, and isolated capital per trade.")
    combos = generate_param_combinations(GRID_SEARCH_SPACE)
    print(f"TOTAL combos to run: {len(combos)}")

    if WARM_CACHE:
        print("WARM_CACHE is True — warming cache sequentially before threaded run (recommended).")
        warm_cache(TICKER_LIST)

    results_df = run_grid_search_parallel(TICKER_LIST, GRID_SEARCH_SPACE, save_detailed=SAVE_DETAILED_TRADES)

    if results_df is not None and not results_df.empty:
        top_by_total = results_df.sort_values(by='total_net_profit', ascending=False).head(10)
        print("\\nTop 10 combos by total_net_profit:")
        print(top_by_total[['total_net_profit', 'overall_roi_pct', 'win_rate', 'total_trades']].to_string(index=False))
        top_by_win = results_df.sort_values(by='win_rate', ascending=False).head(10)
        print("\\nTop 10 combos by win_rate:")
        print(top_by_win[['win_rate', 'total_net_profit', 'overall_roi_pct', 'total_trades']].to_string(index=False))
    else:
        print("No results produced.")

    print("\\nGrid search finished. Results in:", GRID_RESULTS_CSV)
